In [1]:
import pandas as pd
from anomaly_detection import detect_anomaly
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

In [12]:
import os
os.getcwd()

'/home/kpapdac'

In [11]:
#!pip install git+https://github.com/kpapdac/anomaly_detection.git

  Cloning https://github.com/kpapdac/anomaly_detection.git to /var/tmp/pip-req-build-bxgs0tv8
  Running command git clone --filter=blob:none --quiet https://github.com/kpapdac/anomaly_detection.git /var/tmp/pip-req-build-bxgs0tv8
  Resolved https://github.com/kpapdac/anomaly_detection.git to commit 3efc69924bc759091082ba8dfaf52990daf98d6f
  Preparing metadata (setup.py) ... done


In [3]:
donors = pd.read_csv(r'https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/raw/main/numerical%20data/DevNet%20datasets/KDD2014_donors_10feat_nomissing_normalised.csv')

The donors data is taken from KDD Cup 2014 for predicting excitement of projects proposed by K-12 school teachers, in which
exceptionally exciting projects are used as anomalies (5.92% data).

In [4]:
donors.groupby('class')['fully_funded=t'].count()

class
0    582616
1     36710
Name: fully_funded=t, dtype: int64

In [5]:
donors.nunique()

at_least_1_teacher_referred_donor=t                   3
fully_funded=t                                        2
at_least_1_green_donation=0                           3
great_chat=t                                          2
three_or_more_non_teacher_referred_donors=0           3
one_non_teacher_referred_donor_giving_100_plus=t      3
donation_from_thoughtful_donor=t                      3
great_messages_proportion                           102
teacher_referred_count                               82
non_teacher_referred_count                          135
class                                                 2
dtype: int64

Sample stratification

In [6]:
X = donors.iloc[0:5000,0:donors.shape[1]-1].copy().to_numpy()
y = donors.iloc[0:5000,donors.shape[1]-1:donors.shape[1]].copy().to_numpy()
groups = donors.iloc[0:5000,donors.shape[1]-1:donors.shape[1]].copy().to_numpy()
gss = GroupShuffleSplit(n_splits=2, train_size=.7, random_state=42)
X_train_twoclass = []
y_train_twoclass = []
for i, (train_index, test_index) in enumerate(gss.split(X, y,groups)):
    print(f"Fold {i}:")
    print(len(groups[train_index]))
    X_train = X[train_index]
    y_train = y[train_index]
    X_train_twoclass.append(X_train)
    y_train_twoclass.append(y_train)
print(np.concatenate([X_train_twoclass[0],X_train_twoclass[1]]).shape)
X_train_m = np.concatenate([X_train_twoclass[0],X_train_twoclass[1]])
y_train_m = np.concatenate([y_train_twoclass[0],y_train_twoclass[1]])

Fold 0:
4694
Fold 1:
306
(5000, 10)


OneClassSVM, Isolation Forest, Kmeans, Gaussian MM

In [7]:
X = X_train_m.copy()
pred_svm = detect_anomaly.runOneClassSVM().fit_predict(X,X)
pred_if = detect_anomaly.runIsolationForest().fit_predict(X,X)
pred_km = detect_anomaly.runKMeans(2).fit_predict(X,X)
pred_gmm = detect_anomaly.runGaussianMixture(2).fit_predict(X,X)

In [8]:
(pred_svm==1).sum(),(pred_svm==-1).sum(), (pred_if==1).sum(),(pred_if==-1).sum(), (pred_km==1).sum(),(pred_km==0).sum(), (pred_gmm[0]==1).sum(),(pred_gmm[0]==0).sum()

(2512, 2488, 2769, 2231, 1561, 3439, 2210, 2790)

Deep Learning

In [9]:
nn_model = detect_anomaly.NeuralNetwork(D_in=X.shape[0],H=30)
x = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y)
# x = torch.randn(128, 20)
# y = torch.from_numpy(np.repeat(0,128))
X_y = [(x[i],y[i]) for i in range(len(x))]
dataset_ = loader.FeatureClusterDataset(X_y)
train_dataloader = DataLoader(dataset_, batch_size=64)
opt = detect_anomaly.optimizeNN(train_dataloader, nn_model, learning_rate=1e-3, batch_size=64, epochs=5)
opt.train_loop()
opt.iterate()

AttributeError: module 'anomaly_detection.detect_anomaly' has no attribute 'NeuralNetwork'

In [10]:
dir(detect_anomaly)

['IsolationForest',
 'KMeans',
 'OneClassSVM',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'mixture',
 'runGaussianMixture',
 'runIsolationForest',
 'runKMeans',
 'runOneClassSVM']